# tensorboard 可视化

In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer # 独热处理
import pandas as pd

导入数据集，使用sklearn中的load_digits

In [2]:
digits = load_digits()
X = digits.data
y = digits.target
print X.shape
print y.shape
y = LabelBinarizer().fit_transform(y)
print y.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(1797, 64)
(1797,)
(1797, 10)
(1257, 64) (1257, 10)
(540, 64) (540, 10)


In [3]:
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.matmul(inputs, Weights) + biases
            # here to dropout
            Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
            tf.summary.histogram(layer_name + '/Wx_plus_b', Wx_plus_b)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

In [4]:
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32, name='drop_prob')
    xs = tf.placeholder(tf.float32, [None, 64], name='x_input')  # 8x8
    ys = tf.placeholder(tf.float32, [None, 10], name='y_input')

搭建三层神经网络，隐藏层50个神经元

In [5]:
l1 = add_layer(xs, 64, 50, n_layer=1, activation_function=tf.nn.sigmoid) # 这里激活函数用relu报错，why？
prediction = add_layer(l1, 50, 10, n_layer=2, activation_function=tf.nn.softmax)

In [6]:
with tf.name_scope('cross_entropy'):
    # 优化函数选用交叉熵，如果完全相同则为0
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))
    tf.summary.scalar('loss', cross_entropy)
with tf.name_scope('train_step'):
    # 优化算法使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy) # 不断下降交叉熵使其降为0

In [7]:
sess = tf.Session()
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('logs/train', sess.graph)
test_writer = tf.summary.FileWriter('logs/test', sess.graph)
init = tf.global_variables_initializer()
sess.run(init)

In [8]:
for i in range(500):
    # here to determine the keeping probability
    sess.run(train_step, feed_dict={xs: X_train, ys: y_train, keep_prob: 0.5})
    if i % 50 == 0:
        # record loss
        train_result = sess.run(merged, feed_dict={xs: X_train, ys: y_train, keep_prob: 1})
        test_result = sess.run(merged, feed_dict={xs: X_test, ys: y_test, keep_prob: 1})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)

查看对test上的预测结果

In [9]:
result_l1 = sess.run(l1, feed_dict={xs: X_test, keep_prob: 1}) # 隐藏层的计算结果
result_prediction = sess.run(prediction, feed_dict={l1: result_l1, keep_prob: 1}) # 输出层的计算结果

In [10]:
print result_l1.shape
print result_prediction.shape

(540, 50)
(540, 10)


In [11]:
print result_prediction[0] # 可以看到这里的计算结果是概率，并且二号位上概率最大

[  7.83409923e-03   6.79480704e-03   1.05691666e-03   6.48248097e-05
   3.51889059e-03   1.31188659e-02   1.43297412e-03   1.59399621e-02
   9.44342196e-01   5.89638297e-03]


In [12]:
print y_test[0]

[0 0 0 0 0 0 0 0 1 0]


In [13]:
# 这个就是预测结果
sess.run(tf.arg_max(result_prediction, 1))

array([8, 2, 0, 8, 2, 2, 4, 3, 3, 4, 6, 8, 9, 4, 0, 8, 3, 7, 6, 0, 3, 6, 8,
       3, 9, 9, 7, 6, 6, 7, 2, 5, 2, 6, 8, 5, 1, 2, 8, 2, 3, 3, 7, 5, 0, 9,
       2, 0, 2, 9, 4, 0, 6, 8, 7, 7, 7, 8, 9, 6, 3, 5, 1, 3, 5, 6, 3, 1, 0,
       2, 0, 3, 2, 2, 2, 8, 3, 9, 9, 4, 1, 6, 9, 7, 9, 4, 0, 4, 1, 6, 3, 2,
       6, 2, 8, 1, 3, 8, 0, 6, 4, 0, 4, 1, 8, 9, 2, 9, 3, 1, 1, 2, 5, 8, 7,
       8, 3, 3, 7, 3, 2, 1, 7, 8, 3, 5, 5, 3, 0, 5, 3, 0, 6, 2, 2, 2, 4, 0,
       5, 8, 5, 2, 6, 6, 2, 4, 7, 4, 9, 8, 5, 8, 7, 0, 5, 4, 0, 2, 9, 4, 0,
       0, 9, 0, 9, 0, 8, 4, 0, 0, 3, 9, 2, 2, 3, 2, 8, 9, 2, 9, 4, 8, 7, 1,
       5, 5, 2, 9, 6, 3, 8, 4, 6, 8, 5, 5, 4, 7, 1, 0, 7, 4, 1, 0, 7, 8, 5,
       0, 2, 5, 3, 1, 5, 8, 8, 9, 5, 5, 9, 8, 1, 9, 1, 0, 0, 9, 0, 6, 2, 3,
       7, 0, 2, 4, 7, 7, 2, 9, 2, 8, 5, 0, 8, 8, 0, 5, 9, 3, 4, 5, 6, 5, 9,
       6, 1, 6, 9, 2, 1, 6, 5, 0, 4, 1, 6, 9, 7, 6, 2, 9, 1, 7, 1, 3, 8, 3,
       4, 1, 0, 1, 1, 2, 1, 6, 8, 7, 3, 1, 5, 7, 9, 6, 6, 5, 0, 6, 2, 4, 1,
       8, 0,

In [14]:
# 这个是测试值
sess.run(tf.arg_max(y_test, 1))

array([8, 2, 0, 3, 2, 2, 4, 3, 9, 4, 6, 8, 9, 4, 0, 8, 3, 7, 6, 0, 2, 6, 8,
       9, 9, 9, 7, 6, 6, 7, 2, 8, 2, 6, 8, 5, 2, 2, 8, 2, 3, 3, 7, 5, 0, 9,
       2, 0, 2, 9, 4, 0, 6, 8, 7, 4, 7, 8, 9, 6, 8, 6, 1, 9, 5, 6, 3, 1, 0,
       2, 0, 3, 2, 2, 2, 8, 9, 9, 9, 4, 1, 6, 9, 7, 9, 4, 0, 4, 1, 6, 3, 2,
       6, 2, 8, 1, 3, 8, 0, 6, 4, 0, 4, 1, 8, 9, 2, 9, 3, 1, 1, 2, 5, 8, 7,
       8, 9, 3, 7, 3, 2, 1, 7, 9, 3, 5, 5, 3, 4, 5, 8, 0, 6, 2, 2, 2, 4, 6,
       5, 8, 9, 2, 6, 6, 2, 4, 7, 4, 9, 8, 5, 8, 7, 0, 5, 4, 0, 2, 9, 4, 6,
       0, 9, 0, 9, 0, 8, 4, 6, 0, 3, 9, 2, 2, 3, 2, 8, 5, 2, 9, 4, 8, 9, 1,
       5, 5, 2, 9, 6, 3, 8, 4, 6, 8, 5, 5, 4, 7, 1, 0, 7, 4, 1, 0, 7, 6, 5,
       0, 8, 5, 3, 3, 5, 8, 4, 9, 5, 5, 9, 8, 5, 9, 1, 0, 0, 9, 5, 6, 2, 3,
       7, 0, 2, 4, 7, 7, 2, 9, 2, 8, 5, 0, 8, 8, 0, 5, 8, 3, 4, 5, 6, 5, 9,
       6, 1, 6, 9, 2, 1, 6, 9, 0, 4, 1, 6, 9, 7, 6, 2, 9, 9, 7, 1, 3, 8, 3,
       4, 1, 0, 1, 1, 8, 1, 6, 8, 7, 9, 1, 5, 7, 9, 6, 6, 5, 0, 6, 2, 4, 1,
       8, 5,

In [15]:
# 结果正确为1，反之0
sess.run(tf.equal(tf.arg_max(result_prediction, 1), tf.arg_max(y_test, 1)))

array([ True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True, False, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,

In [16]:
correct_prediction = tf.equal(tf.argmax(result_prediction,1), tf.argmax(y_test,1))
sess.run(tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))

0.88518518

准确率为0.896

运行结束，终端运行tensorboard --logdir='logs/'